Created because I hate Excel so much. -K Droubi 4 April 2021

Designed to run with slightly modified version of MassHunter offline cps table export.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import linregress
import os

df = pd.read_csv ('TE_test/cps_table1_mod.csv')
#df

In [2]:
#Functions

def cps_parser(df):
    cps_dict = {}
    
    cps_columnIDX = ['Sample Name']
    cps_rsd_columnIDX = ['Sample Name']
    for idx in df:
        #print(df.loc[0, idx])
        if str(df.loc[0, idx]) == 'CPS':
            cps_columnIDX.append(idx)
        if str(df.loc[0, idx]) == 'CPS RSD':
            cps_rsd_columnIDX.append(idx)
    #cps_columnIDX
    df_cps_crop = df[cps_columnIDX]
    df_cps_rsd_crop = df[cps_rsd_columnIDX]
    df_cps_rsd_crop.columns = cps_columnIDX
    #df_cps_rsd_crop

    cps_NoGas_columnIDX = ['Sample Name']
    cps_O2_columnIDX = ['Sample Name']

    for idx in df_cps_crop:
        if 'No' in idx:
            cps_NoGas_columnIDX.append(idx)
        if 'O2' in idx:
            cps_O2_columnIDX.append(idx)
            
    cps_dict['df_nogas_cps'] = df_cps_crop[cps_NoGas_columnIDX]
    cps_dict['df_O2_cps'] = df_cps_crop[cps_O2_columnIDX]
    cps_dict['df_nogas_cps_rsd'] = df_cps_rsd_crop[cps_NoGas_columnIDX]
    cps_dict['df_O2_cps_rsd'] = df_cps_rsd_crop[cps_O2_columnIDX] 
    
    for i in cps_dict:
        df2 = cps_dict[i].loc[1:]
        for idx in df2:
            if idx == 'Sample Name':
                  continue
            df2[idx] = pd.to_numeric(df2[idx])
        cps_dict[i] = df2
    
    try:
        df_t1 = cps_dict['df_nogas_cps']
        df_t1_new = df_t1.rename(columns={'207  Pb  [ No Gas ] .1': '207  Pb  [ No Gas ] ', '208  Pb  [ No Gas ] .1': '208  Pb  [ No Gas ] '})
        cps_dict['df_nogas_cps'] = df_t1_new
        df_t2 = cps_dict['df_nogas_cps_rsd']
        df_t2_new = df_t2.rename(columns={'207  Pb  [ No Gas ] .1': '207  Pb  [ No Gas ] ', '208  Pb  [ No Gas ] .1': '208  Pb  [ No Gas ] '})
        cps_dict['df_nogas_cps_rsd'] = df_t2_new
    except:
        pass
    
    return cps_dict

def ratio_cps_vs_istd(cps_df, run_type = 'No Gas'):
    #cps_df you want from cps_parser function
    #run_type can be: 'No Gas' or 'O2', for now...
    
    #Define which masses get which ISTD's here:
    istd_is_In = ['89', '139', '140', '141', '146', '147', '153', '157', '159', '163', '165', '166', '169', '172', '175']
    istd_is_Bi = ['206', '207', '208', '232', '238']

    #Math which divides CPS of element by ISTD
    df2 = cps_df
    
    if run_type == 'No Gas':
        
        for idx in istd_is_In:
            df2_ratio = df2.apply(lambda x: x / df2['115  In ( ISTD )  [ No Gas ] '] if idx in x.name else x)
            df2 = df2_ratio
        for idx in istd_is_Bi:
            df2_ratio = df2.apply(lambda x: x / df2['209  Bi ( ISTD )  [ No Gas ] '] if idx in x.name else x)
            df2 = df2_ratio
    
    if run_type == 'O2':
        for idx in istd_is_In:
            df2_ratio = df2.apply(lambda x: x / df2['115 -> 115  In ( ISTD )  [ O2 ] '] if idx in x.name else x)
            df2 = df2_ratio
        for idx in istd_is_Bi:
            df2_ratio = df2.apply(lambda x: x / df2['209 -> 209  Bi ( ISTD )  [ O2 ] '] if idx in x.name else x)
            df2 = df2_ratio
    
    return df2

def calibration(cps_ratios_df, expected_values_df, num_calstd = 9): ###wrong, needs to only use :9
    slope_dict = {}
    intercept_dict = {}
    r_value_dict = {}
    
    #df_test1 = cps_ratios_df.loc[:9, :]
    for idx in cps_ratios_df:
        if idx == 'Sample Name':
            continue
        #print(idx)
        
        x = expected_values_df[idx][:9]
        #print(x)
        #print('DEBUG')
        #break
        y = cps_ratios_df[idx][:9]
        slope, intercept, r_value, p_value, std_err = linregress(x,y)

        slope_dict[idx] = slope
        intercept_dict[idx] = cps_ratios_df[idx][1]
        r_value_dict[idx] = r_value 
        
    test_list = [slope_dict, intercept_dict, r_value_dict ]
    output_df = pd.DataFrame(test_list, index = ['slope', 'intercept', 'r-value'])
    
    return output_df

def calc_conc(cps_ratios_df, calibration_df):

    df_test3 = cps_ratios_df
    cal = calibration_df
    
    conc_dict = {}
    for key in df_test3:
        if key == 'Sample Name':
                conc_dict[key] = df_test3[key]
                continue
        #print(key)
        y = df_test3.loc[:, key]
        x = (y - cal.loc['intercept', key]) / cal.loc['slope', key]
        conc_dict[key] = x
    output = pd.DataFrame(conc_dict)
    
    return output

def cps_to_conc(cps_df, run_type, expected_values_df):
    
    cps_ratios_df = ratio_cps_vs_istd(cps_df, run_type)
    calibration_df = calibration(cps_ratios_df, expected_values_df)
    output = calc_conc(cps_ratios_df, calibration_df)
    
    return output



In [3]:
cps_dict = cps_parser(df)

<ipython-input-2-69654cbe0950>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[idx] = pd.to_numeric(df2[idx])


In [4]:
cps_ratios_dict = {}
for key in cps_dict:
    if 'no' in key:
        cps_ratios_dict[key] = ratio_cps_vs_istd(cps_dict[key],'No Gas')
    if 'O2' in key:
        cps_ratios_dict[key] = ratio_cps_vs_istd(cps_dict[key],'O2')
cps_ratios_dict.keys()

dict_keys(['df_nogas_cps', 'df_O2_cps', 'df_nogas_cps_rsd', 'df_O2_cps_rsd'])

In [5]:
levels = pd.read_csv ('TE_test/levels_TE3.csv')
levels
#levels.columns

,Unnamed: 0,Unnamed: 1,Unnamed: 2,89 Y [ No Gas ],139 La [ No Gas ],140 Ce [ No Gas ],141 Pr [ No Gas ],146 Nd [ No Gas ],147 Sm [ No Gas ],153 Eu [ No Gas ],...,207 -> 223 Pb [ O2 ],208 -> 208 Pb [ O2 ],208 -> 224 Pb [ O2 ],232 -> 232 Th [ O2 ],232 -> 248 Th [ O2 ],238 -> 238 U [ O2 ],238 -> 254 U [ O2 ],72 -> 72 Ge ( ISTD ) [ O2 ],115 -> 115 In ( ISTD ) [ O2 ],209 -> 209 Bi ( ISTD ) [ O2 ]
0,2% HNO3,Level1,[ppb],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
1,BCC5Yb0.0001,Level2,[ppb],0.001050,0.001550,0.003150,0.000360,0.001050,0.000240,0.000050,...,0.000850,0.000850,0.000850,0.000530,0.000530,0.000140,0.000140,NaN,NaN,NaN
2,BCC5Yb0.001,Level3,[ppb],0.010500,0.015500,0.031500,0.003550,0.010500,0.002350,0.000500,...,0.008500,0.008500,0.008500,0.005250,0.005250,0.001350,0.001350,NaN,NaN,NaN
3,BCC5Yb0.005,Level4,[ppb],0.052500,0.077500,0.157500,0.017800,0.052500,0.011800,0.002500,...,0.042500,0.042500,0.042500,0.026300,0.026300,0.006800,0.006800,NaN,NaN,NaN
4,BCC5Yb0.01,Level5,[ppb],0.105000,0.155000,0.315000,0.035500,0.105000,0.023500,0.005000,...,0.085000,0.085000,0.085000,0.052500,0.052500,0.013500,0.013500,NaN,NaN,NaN
5,BCC5Yb0.02,Level6,[ppb],0.210000,0.310000,0.630000,0.071000,0.210000,0.047000,0.010000,...,0.170000,0.170000,0.170000,0.105000,0.105000,0.027000,0.027000,NaN,NaN,NaN
6,BCC5Yb0.05,Level7,[ppb],0.525000,0.775000,1.575000,0.178000,0.525000,0.118000,0.025000,...,0.425000,0.425000,0.425000,0.263000,0.263000,0.068000,0.068000,NaN,NaN,NaN
7,BCC5Yb0.1,Level8,[ppb],1.050000,1.550000,3.150000,0.355000,1.050000,0.235000,0.050000,...,0.850000,0.850000,0.850000,0.525000,0.525000,0.135000,0.135000,NaN,NaN,NaN
8,BCC5Yb1,Level9,[ppb],10.500000,15.500000,31.500000,3.550000,10.500000,2.350000,0.500000,...,8.500000,8.500000,8.500000,5.250000,5.250000,1.350000,1.350000,NaN,NaN,NaN
9,S-TTN (Y9E5),Sample1,[ppb],3.007703,0.534440,2.315320,0.405420,2.031878,0.571662,0.117652,...,0.066145,0.066145,0.066145,0.279066,0.279066,0.275595,0.275595,NaN,NaN,NaN


In [6]:
cps_ratios_dict['df_nogas_cps']['89  Y  [ No Gas ] '][1]
#levels.columns

2.2329716331528544e-05

In [7]:
calibration(cps_ratios_dict['df_nogas_cps'], levels)

,89 Y [ No Gas ],139 La [ No Gas ],140 Ce [ No Gas ],141 Pr [ No Gas ],146 Nd [ No Gas ],147 Sm [ No Gas ],153 Eu [ No Gas ],157 Gd [ No Gas ],159 Tb [ No Gas ],163 Dy [ No Gas ],...,175 Lu [ No Gas ],206 Pb [ No Gas ],207 Pb [ No Gas ],208 Pb [ No Gas ],232 Th [ No Gas ],238 U [ No Gas ],7 Li ( ISTD ) [ No Gas ],72 Ge ( ISTD ) [ No Gas ],115 In ( ISTD ) [ No Gas ],209 Bi ( ISTD ) [ No Gas ]
slope,1.009648,1.107219,1.030283,1.189506,0.208750,0.174887,0.635156,0.218444,1.271989,0.292566,...,1.198918e+00,0.362331,0.314995,0.781341,1.434329,1.389941,NaN,NaN,NaN,NaN
intercept,0.000022,0.000300,0.000480,0.000092,0.000029,0.000002,0.000011,0.000014,0.000010,0.000003,...,5.668313e-07,0.001488,0.001211,0.002957,0.000051,0.000007,117619.91,93937.88,582183.84,384518.54
r-value,0.999937,0.999933,0.999921,0.999931,0.999901,0.999904,0.999898,0.999903,0.999913,0.999917,...,9.999432e-01,0.999915,0.999917,0.999932,0.999940,0.999910,NaN,NaN,NaN,NaN


In [8]:
calibration(cps_ratios_dict['df_O2_cps'], levels)

,89 -> 105 Y [ O2 ],140 -> 140 Ce [ O2 ],139 -> 155 La [ O2 ],140 -> 156 Ce [ O2 ],141 -> 157 Pr [ O2 ],146 -> 162 Nd [ O2 ],147 -> 163 Sm [ O2 ],153 -> 169 Eu [ O2 ],157 -> 173 Gd [ O2 ],159 -> 175 Tb [ O2 ],...,207 -> 223 Pb [ O2 ],208 -> 208 Pb [ O2 ],208 -> 224 Pb [ O2 ],232 -> 232 Th [ O2 ],232 -> 248 Th [ O2 ],238 -> 238 U [ O2 ],238 -> 254 U [ O2 ],72 -> 72 Ge ( ISTD ) [ O2 ],115 -> 115 In ( ISTD ) [ O2 ],209 -> 209 Bi ( ISTD ) [ O2 ]
slope,1.019453,0.000053,1.086926,1.046463,3.793693e-06,0.211041,6.041105e-07,2.911813e-07,0.192661,4.249335e-06,...,0.001393,0.749758,0.003318,0.000005,0.401165,0.000003,0.001174,NaN,NaN,NaN
intercept,0.000065,0.000003,0.000218,0.000366,2.840565e-10,0.000033,8.328536e-11,7.612714e-12,0.000008,5.681130e-11,...,0.000013,0.002893,0.000021,0.000002,0.000022,0.000002,0.000002,36218.98,296665.91,207873.71
r-value,0.999992,0.999776,0.999982,0.999989,9.998722e-01,0.999904,9.998800e-01,9.998186e-01,0.999922,9.999067e-01,...,0.999864,0.999896,0.999880,0.966604,0.999900,0.691321,0.999797,NaN,NaN,NaN


In [9]:
# concentration [ppb] = ([cps / ISTD value] - intercept) / slope

key = '89  Y  [ No Gas ] '
df_test3 = cps_ratios_dict['df_nogas_cps']
nogas_cal = calibration(cps_ratios_dict['df_nogas_cps'], levels)
y = df_test3.loc[:, key]
x = (y - nogas_cal.loc['intercept', key]) / nogas_cal.loc['slope', key]

df_test3.index

RangeIndex(start=1, stop=15, step=1)

In [10]:
calc_conc(df_test3, nogas_cal)

,Sample Name,89 Y [ No Gas ],139 La [ No Gas ],140 Ce [ No Gas ],141 Pr [ No Gas ],146 Nd [ No Gas ],147 Sm [ No Gas ],153 Eu [ No Gas ],157 Gd [ No Gas ],159 Tb [ No Gas ],...,175 Lu [ No Gas ],206 Pb [ No Gas ],207 Pb [ No Gas ],208 Pb [ No Gas ],232 Th [ No Gas ],238 U [ No Gas ],7 Li ( ISTD ) [ No Gas ],72 Ge ( ISTD ) [ No Gas ],115 In ( ISTD ) [ No Gas ],209 Bi ( ISTD ) [ No Gas ]
1,2% HNO3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
2,BCC5-Yb0.0001,0.001233,0.001751,0.003469,0.000342,0.001291,0.000327,0.000061,0.000144,0.000039,...,0.000022,-0.001124,-0.001016,-0.000912,0.000583,0.000186,NaN,NaN,NaN,NaN
3,BCC5-Yb0.001,0.011415,0.017174,0.034695,0.003860,0.011923,0.002383,0.000554,0.002181,0.000389,...,0.000170,0.007504,0.007631,0.007441,0.005741,0.001509,NaN,NaN,NaN,NaN
4,BCC5-Yb0.005,0.058006,0.085714,0.174674,0.019738,0.059393,0.013236,0.002818,0.011195,0.001991,...,0.000864,0.046384,0.045762,0.045115,0.029094,0.007644,NaN,NaN,NaN,NaN
5,BCC5-Yb0.01,0.118675,0.174349,0.355196,0.040236,0.122203,0.026899,0.005726,0.023016,0.003952,...,0.001790,0.095744,0.095012,0.094152,0.058822,0.015701,NaN,NaN,NaN,NaN
6,BCC5-Yb0.02,0.234672,0.344751,0.702493,0.079310,0.239763,0.053542,0.011205,0.044932,0.007941,...,0.003533,0.192080,0.191989,0.188510,0.116500,0.030684,NaN,NaN,NaN,NaN
7,BCC5-Yb0.05,0.567569,0.836961,1.704210,0.192791,0.579869,0.129091,0.027784,0.110587,0.019136,...,0.008627,0.464810,0.465223,0.461087,0.282957,0.074582,NaN,NaN,NaN,NaN
8,BCC5-Yb0.1,1.170925,1.734009,3.556909,0.397789,1.200704,0.268352,0.057236,0.228341,0.039777,...,0.017672,0.961584,0.960467,0.949245,0.583852,0.153515,NaN,NaN,NaN,NaN
9,BCC5-Yb1,10.514502,15.521245,31.545081,3.555020,10.518190,2.353812,0.500840,2.003286,0.350500,...,0.155222,8.512304,8.512132,8.510655,5.256892,1.352234,NaN,NaN,NaN,NaN
10,S-TTN-Y9E5,2.901404,0.501689,2.170544,0.390856,1.937399,0.499173,0.115171,0.410676,0.100017,...,0.025298,0.062647,0.062696,0.061787,0.270882,0.275015,NaN,NaN,NaN,NaN


In [11]:
calc_conc(cps_ratios_dict['df_O2_cps'], calibration(cps_ratios_dict['df_O2_cps'], levels))

,Sample Name,89 -> 105 Y [ O2 ],140 -> 140 Ce [ O2 ],139 -> 155 La [ O2 ],140 -> 156 Ce [ O2 ],141 -> 157 Pr [ O2 ],146 -> 162 Nd [ O2 ],147 -> 163 Sm [ O2 ],153 -> 169 Eu [ O2 ],157 -> 173 Gd [ O2 ],...,207 -> 223 Pb [ O2 ],208 -> 208 Pb [ O2 ],208 -> 224 Pb [ O2 ],232 -> 232 Th [ O2 ],232 -> 248 Th [ O2 ],238 -> 238 U [ O2 ],238 -> 254 U [ O2 ],72 -> 72 Ge ( ISTD ) [ O2 ],115 -> 115 In ( ISTD ) [ O2 ],209 -> 209 Bi ( ISTD ) [ O2 ]
1,2% HNO3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
2,BCC5-Yb0.0001,0.001085,-0.042981,0.001534,0.003271,0.000402,0.001278,0.000236,0.000139,0.000225,...,-0.003429,-0.000837,-0.001430,0.628266,0.000582,-0.556621,-0.001352,NaN,NaN,NaN
3,BCC5-Yb0.001,0.010030,0.083843,0.016282,0.032758,0.003804,0.011817,0.002552,0.000641,0.002224,...,0.001246,0.007676,0.005923,-0.307578,0.006171,0.005704,0.001421,NaN,NaN,NaN
4,BCC5-Yb0.005,0.050947,0.172511,0.082078,0.164818,0.020750,0.060215,0.013596,0.002729,0.011442,...,0.048632,0.047088,0.040327,-0.307578,0.030370,-0.556621,0.007055,NaN,NaN,NaN
5,BCC5-Yb0.01,0.102505,0.148558,0.165952,0.333436,0.041254,0.120550,0.027865,0.005860,0.022834,...,0.104222,0.095449,0.093893,0.951745,0.061492,0.591439,0.012508,NaN,NaN,NaN
6,BCC5-Yb0.02,0.206652,0.748076,0.333600,0.671304,0.083554,0.242082,0.055322,0.011387,0.045198,...,0.172604,0.196561,0.189541,-0.307578,0.122307,-0.556621,0.053276,NaN,NaN,NaN
7,BCC5-Yb0.05,0.492303,1.983023,0.797649,1.598435,0.201710,0.582445,0.134210,0.029040,0.109870,...,0.443091,0.468402,0.473252,0.333155,0.292449,-0.556621,0.078517,NaN,NaN,NaN
8,BCC5-Yb0.1,1.015613,3.665349,1.647433,3.303656,0.412327,1.198437,0.271587,0.059541,0.225394,...,0.990183,0.974094,0.980007,0.961685,0.600910,0.600502,0.139891,NaN,NaN,NaN
9,BCC5-Yb1,10.494587,31.552242,15.511060,31.517014,3.557230,10.518518,2.354841,0.501097,2.003174,...,8.509022,8.513808,8.512018,5.344958,5.259783,1.148271,1.354233,NaN,NaN,NaN
10,S-TTN-Y9E5,2.498332,2.191874,0.481461,2.051365,0.407676,1.951054,0.508307,0.117664,0.455095,...,0.075570,0.063633,0.066411,-0.307578,0.275188,-0.556621,0.290696,NaN,NaN,NaN


At this point in the code:

You should be able to create

1) dataframes of No Gas and O2 CPS and CPS_RSD

2) dataframes of CPS / ISTD for No Gas and O2

3) dataframes of slope, intercept, and r-value for No Gas and O2 matched with expected calibration values

4) Calculate concentrations using slope and intercept values. //slightly off from MassHunter output :(

To Do:

Write out function for oxide correction (probably using lambda correction...)

In [12]:
cps_to_conc(cps_dict['df_O2_cps'], 'O2', levels)

,Sample Name,89 -> 105 Y [ O2 ],140 -> 140 Ce [ O2 ],139 -> 155 La [ O2 ],140 -> 156 Ce [ O2 ],141 -> 157 Pr [ O2 ],146 -> 162 Nd [ O2 ],147 -> 163 Sm [ O2 ],153 -> 169 Eu [ O2 ],157 -> 173 Gd [ O2 ],...,207 -> 223 Pb [ O2 ],208 -> 208 Pb [ O2 ],208 -> 224 Pb [ O2 ],232 -> 232 Th [ O2 ],232 -> 248 Th [ O2 ],238 -> 238 U [ O2 ],238 -> 254 U [ O2 ],72 -> 72 Ge ( ISTD ) [ O2 ],115 -> 115 In ( ISTD ) [ O2 ],209 -> 209 Bi ( ISTD ) [ O2 ]
1,2% HNO3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
2,BCC5-Yb0.0001,0.001085,-0.042981,0.001534,0.003271,0.000402,0.001278,0.000236,0.000139,0.000225,...,-0.003429,-0.000837,-0.001430,0.628266,0.000582,-0.556621,-0.001352,NaN,NaN,NaN
3,BCC5-Yb0.001,0.010030,0.083843,0.016282,0.032758,0.003804,0.011817,0.002552,0.000641,0.002224,...,0.001246,0.007676,0.005923,-0.307578,0.006171,0.005704,0.001421,NaN,NaN,NaN
4,BCC5-Yb0.005,0.050947,0.172511,0.082078,0.164818,0.020750,0.060215,0.013596,0.002729,0.011442,...,0.048632,0.047088,0.040327,-0.307578,0.030370,-0.556621,0.007055,NaN,NaN,NaN
5,BCC5-Yb0.01,0.102505,0.148558,0.165952,0.333436,0.041254,0.120550,0.027865,0.005860,0.022834,...,0.104222,0.095449,0.093893,0.951745,0.061492,0.591439,0.012508,NaN,NaN,NaN
6,BCC5-Yb0.02,0.206652,0.748076,0.333600,0.671304,0.083554,0.242082,0.055322,0.011387,0.045198,...,0.172604,0.196561,0.189541,-0.307578,0.122307,-0.556621,0.053276,NaN,NaN,NaN
7,BCC5-Yb0.05,0.492303,1.983023,0.797649,1.598435,0.201710,0.582445,0.134210,0.029040,0.109870,...,0.443091,0.468402,0.473252,0.333155,0.292449,-0.556621,0.078517,NaN,NaN,NaN
8,BCC5-Yb0.1,1.015613,3.665349,1.647433,3.303656,0.412327,1.198437,0.271587,0.059541,0.225394,...,0.990183,0.974094,0.980007,0.961685,0.600910,0.600502,0.139891,NaN,NaN,NaN
9,BCC5-Yb1,10.494587,31.552242,15.511060,31.517014,3.557230,10.518518,2.354841,0.501097,2.003174,...,8.509022,8.513808,8.512018,5.344958,5.259783,1.148271,1.354233,NaN,NaN,NaN
10,S-TTN-Y9E5,2.498332,2.191874,0.481461,2.051365,0.407676,1.951054,0.508307,0.117664,0.455095,...,0.075570,0.063633,0.066411,-0.307578,0.275188,-0.556621,0.290696,NaN,NaN,NaN


In [13]:
cps_dict['df_nogas_cps'].columns

Index(['Sample Name', '89  Y  [ No Gas ] ', '139  La  [ No Gas ] ',
       '140  Ce  [ No Gas ] ', '141  Pr  [ No Gas ] ', '146  Nd  [ No Gas ] ',
       '147  Sm  [ No Gas ] ', '153  Eu  [ No Gas ] ', '157  Gd  [ No Gas ] ',
       '159  Tb  [ No Gas ] ', '163  Dy  [ No Gas ] ', '165  Ho  [ No Gas ] ',
       '166  Er  [ No Gas ] ', '169  Tm  [ No Gas ] ', '172  Yb  [ No Gas ] ',
       '175  Lu  [ No Gas ] ', '206  Pb  [ No Gas ] ', '207  Pb  [ No Gas ] ',
       '208  Pb  [ No Gas ] ', '232  Th  [ No Gas ] ', '238  U  [ No Gas ] ',
       '7  Li ( ISTD )  [ No Gas ] ', '72  Ge ( ISTD )  [ No Gas ] ',
       '115  In ( ISTD )  [ No Gas ] ', '209  Bi ( ISTD )  [ No Gas ] '],
      dtype='object')

In [14]:
# Oxide Correction
#example test
# Using m/z = 163

#cps_156oxide = cps_140_true * oxide_percentage
#cps_156_total = cps_156oxide + cps_156_true

#cps_156_true = cps_156_total - cps_156oxide

#This value needs to be "corrected" within the cps dataframe, NOT the ratio
oxide_percentage = 0.0136
df8 = cps_dict['df_nogas_cps']

oxide_cps_dict = {
    '157': cps_dict['df_nogas_cps']['141  Pr  [ No Gas ] '],
    '163': cps_dict['df_nogas_cps']['147  Sm  [ No Gas ] '],
    '165': cps_dict['df_nogas_cps']['147  Sm  [ No Gas ] '] * (13.8/15), # 149/147 Sm ratio 
    '159': cps_dict['df_nogas_cps']['146  Nd  [ No Gas ] '] * (12.2/17.2), # 143/146 Nd ratio 
    '166': cps_dict['df_nogas_cps']['146  Nd  [ No Gas ] '] * (5.62/17.2), # 150/146 Nd ratio 
    '169': cps_dict['df_nogas_cps']['153  Eu  [ No Gas ] '],
    '172': cps_dict['df_nogas_cps']['157  Gd  [ No Gas ] '] * (20.5/15.7)
}
mass_to_oxide_correct = ['163'] #This needs to be determined by looking at graphs!

oxide_cps_test = cps_dict['df_nogas_cps'] [ '147  Sm  [ No Gas ] ']
df8_new = df8.apply(lambda x: (x - (oxide_cps_test * oxide_percentage) ) if '163' in x.name else x)
#oxide_cps
#df8_new
df8_new['163  Dy  [ No Gas ] ']
#df_z = cps_to_conc(df8_new, 'No Gas', levels)
#df_z['163  Dy  [ No Gas ] ']

1          1.656400
2         39.868488
3        387.326040
4       1852.731208
5       3776.720600
6       7444.955304
7      17892.355896
8      36751.049048
9     331802.080000
10    117308.213784
11      6416.419840
12    647675.373888
13      1340.029368
14     12677.807520
Name: 163  Dy  [ No Gas ] , dtype: float64

In [15]:
df_old = cps_dict['df_nogas_cps']
df_mod = df_old
for key in oxide_cps_dict:
    
    oxide_cps = oxide_cps_dict[key]
    df_new = df_mod.apply(lambda x: (x - (oxide_cps * oxide_percentage) ) if key in x.name else x)
    
    df_mod = df_new

df_old = cps_to_conc(df_old, 'No Gas', levels)
df_old_O2 = cps_to_conc(cps_dict['df_O2_cps'], 'O2', levels)
df_mod = cps_to_conc(df_mod, 'No Gas', levels)
#df_old['163  Dy  [ No Gas ] ']
df_old_O2

,Sample Name,89 -> 105 Y [ O2 ],140 -> 140 Ce [ O2 ],139 -> 155 La [ O2 ],140 -> 156 Ce [ O2 ],141 -> 157 Pr [ O2 ],146 -> 162 Nd [ O2 ],147 -> 163 Sm [ O2 ],153 -> 169 Eu [ O2 ],157 -> 173 Gd [ O2 ],...,207 -> 223 Pb [ O2 ],208 -> 208 Pb [ O2 ],208 -> 224 Pb [ O2 ],232 -> 232 Th [ O2 ],232 -> 248 Th [ O2 ],238 -> 238 U [ O2 ],238 -> 254 U [ O2 ],72 -> 72 Ge ( ISTD ) [ O2 ],115 -> 115 In ( ISTD ) [ O2 ],209 -> 209 Bi ( ISTD ) [ O2 ]
1,2% HNO3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
2,BCC5-Yb0.0001,0.001085,-0.042981,0.001534,0.003271,0.000402,0.001278,0.000236,0.000139,0.000225,...,-0.003429,-0.000837,-0.001430,0.628266,0.000582,-0.556621,-0.001352,NaN,NaN,NaN
3,BCC5-Yb0.001,0.010030,0.083843,0.016282,0.032758,0.003804,0.011817,0.002552,0.000641,0.002224,...,0.001246,0.007676,0.005923,-0.307578,0.006171,0.005704,0.001421,NaN,NaN,NaN
4,BCC5-Yb0.005,0.050947,0.172511,0.082078,0.164818,0.020750,0.060215,0.013596,0.002729,0.011442,...,0.048632,0.047088,0.040327,-0.307578,0.030370,-0.556621,0.007055,NaN,NaN,NaN
5,BCC5-Yb0.01,0.102505,0.148558,0.165952,0.333436,0.041254,0.120550,0.027865,0.005860,0.022834,...,0.104222,0.095449,0.093893,0.951745,0.061492,0.591439,0.012508,NaN,NaN,NaN
6,BCC5-Yb0.02,0.206652,0.748076,0.333600,0.671304,0.083554,0.242082,0.055322,0.011387,0.045198,...,0.172604,0.196561,0.189541,-0.307578,0.122307,-0.556621,0.053276,NaN,NaN,NaN
7,BCC5-Yb0.05,0.492303,1.983023,0.797649,1.598435,0.201710,0.582445,0.134210,0.029040,0.109870,...,0.443091,0.468402,0.473252,0.333155,0.292449,-0.556621,0.078517,NaN,NaN,NaN
8,BCC5-Yb0.1,1.015613,3.665349,1.647433,3.303656,0.412327,1.198437,0.271587,0.059541,0.225394,...,0.990183,0.974094,0.980007,0.961685,0.600910,0.600502,0.139891,NaN,NaN,NaN
9,BCC5-Yb1,10.494587,31.552242,15.511060,31.517014,3.557230,10.518518,2.354841,0.501097,2.003174,...,8.509022,8.513808,8.512018,5.344958,5.259783,1.148271,1.354233,NaN,NaN,NaN
10,S-TTN-Y9E5,2.498332,2.191874,0.481461,2.051365,0.407676,1.951054,0.508307,0.117664,0.455095,...,0.075570,0.063633,0.066411,-0.307578,0.275188,-0.556621,0.290696,NaN,NaN,NaN


In [16]:
df_mod

,Sample Name,89 Y [ No Gas ],139 La [ No Gas ],140 Ce [ No Gas ],141 Pr [ No Gas ],146 Nd [ No Gas ],147 Sm [ No Gas ],153 Eu [ No Gas ],157 Gd [ No Gas ],159 Tb [ No Gas ],...,175 Lu [ No Gas ],206 Pb [ No Gas ],207 Pb [ No Gas ],208 Pb [ No Gas ],232 Th [ No Gas ],238 U [ No Gas ],7 Li ( ISTD ) [ No Gas ],72 Ge ( ISTD ) [ No Gas ],115 In ( ISTD ) [ No Gas ],209 Bi ( ISTD ) [ No Gas ]
1,2% HNO3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
2,BCC5-Yb0.0001,0.001233,0.001751,0.003469,0.000342,0.001291,0.000327,0.000061,0.000137,0.000039,...,0.000022,-0.001124,-0.001016,-0.000912,0.000583,0.000186,NaN,NaN,NaN,NaN
3,BCC5-Yb0.001,0.011415,0.017174,0.034695,0.003860,0.011923,0.002383,0.000554,0.002182,0.000388,...,0.000170,0.007504,0.007631,0.007441,0.005741,0.001509,NaN,NaN,NaN,NaN
4,BCC5-Yb0.005,0.058006,0.085714,0.174674,0.019738,0.059393,0.013236,0.002818,0.011206,0.001991,...,0.000864,0.046384,0.045762,0.045115,0.029094,0.007644,NaN,NaN,NaN,NaN
5,BCC5-Yb0.01,0.118675,0.174349,0.355196,0.040236,0.122203,0.026899,0.005726,0.023068,0.003946,...,0.001790,0.095744,0.095012,0.094152,0.058822,0.015701,NaN,NaN,NaN,NaN
6,BCC5-Yb0.02,0.234672,0.344751,0.702493,0.079310,0.239763,0.053542,0.011205,0.044970,0.007938,...,0.003533,0.192080,0.191989,0.188510,0.116500,0.030684,NaN,NaN,NaN,NaN
7,BCC5-Yb0.05,0.567569,0.836961,1.704210,0.192791,0.579869,0.129091,0.027784,0.110885,0.019127,...,0.008627,0.464810,0.465223,0.461087,0.282957,0.074582,NaN,NaN,NaN,NaN
8,BCC5-Yb0.1,1.170925,1.734009,3.556909,0.397789,1.200704,0.268352,0.057236,0.228982,0.039765,...,0.017672,0.961584,0.960467,0.949245,0.583852,0.153515,NaN,NaN,NaN,NaN
9,BCC5-Yb1,10.514502,15.521245,31.545081,3.555020,10.518190,2.353812,0.500840,2.003351,0.350497,...,0.155222,8.512304,8.512132,8.510655,5.256892,1.352234,NaN,NaN,NaN,NaN
10,S-TTN-Y9E5,2.901404,0.501689,2.170544,0.390856,1.937399,0.499173,0.115171,0.439503,0.101785,...,0.025298,0.062647,0.062696,0.061787,0.270882,0.275015,NaN,NaN,NaN,NaN


In [17]:
#levels.loc[:, ]
levels_mod = levels.drop('Unnamed: 1', 1)
levels_mod = levels_mod.drop('Unnamed: 2', 1)
#levels_mod

In [18]:
#export to csv
excel_writer = 'TE3_python_conc.xlsx'

with pd.ExcelWriter('TE3_python_conc2.xlsx') as writer:
    df_old.to_excel(writer, sheet_name = 'KD conc', index = False)
    df_old_O2.to_excel(writer, sheet_name = 'KD conc O2', index = False)
    df_mod.to_excel(writer, sheet_name = 'KD oxide_correction', index = False)
    levels_mod.to_excel(writer, sheet_name = 'KD expected conc', index = False)

In [19]:
df8['163  Dy  [ No Gas ] ']

1          1.67
2         40.34
3        390.69
4       1871.22
5       3813.75
6       7519.58
7      18068.76
8      37118.08
9     335085.12
10    117999.03
11      6454.27
12    651455.18
13      1347.80
14     12753.38
Name: 163  Dy  [ No Gas ] , dtype: float64

In [20]:
# This shows the counts that have been subtracted due to oxide correction
df8['163  Dy  [ No Gas ] '] - df8_new['163  Dy  [ No Gas ] ']

1        0.013600
2        0.471512
3        3.363960
4       18.488792
5       37.029400
6       74.624696
7      176.404104
8      367.030952
9     3283.040000
10     690.816216
11      37.850160
12    3779.806112
13       7.770632
14      75.572480
Name: 163  Dy  [ No Gas ] , dtype: float64

In [21]:
df_n = cps_to_conc(cps_dict['df_nogas_cps'], 'No Gas', levels)
df_n[ '147  Sm  [ No Gas ] ']

1     0.000000
2     0.000327
3     0.002383
4     0.013236
5     0.026899
6     0.053542
7     0.129091
8     0.268352
9     2.353812
10    0.499173
11    0.027953
12    2.779555
13    0.005683
14    0.055487
Name: 147  Sm  [ No Gas ] , dtype: float64

In [22]:
old_163 = df_n['163  Dy  [ No Gas ] ']
new_163 = df_z['163  Dy  [ No Gas ] '] 

NameError: name 'df_z' is not defined

In [ ]:
#corrected - uncorrected ... would be expected to be negative
# This shows the change in concentration

# does correction cause an increase in concentration, due to changing calibration curves?

new_163 - old_163

In [ ]:
2.772916 * oxide_percentage

In [ ]:
3.889317 - 0.0377116576

In [ ]:
3.889317 + 0.0377116576

In [ ]:
3.905033